# Based on paper "First Order Motion Model for Image Animation"

**Clone repository**

**Mount your Google drive folder on Colab**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

In [2]:
cd /content/gdrive/My Drive/Colab Notebooks/first-order-model

/content/gdrive/My Drive/Colab Notebooks/first-order-model


**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

#Load image and video
source_image = imageio.imread('/content/gdrive/My Drive/Colab Data/first-order-motion-model/01.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/Colab Data/first-order-motion-model/clip.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

#Display result
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani   

#Show result
HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [ ]:
#Create a model and load checkpoints
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Colab Data/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [ ]:
#Perform image animation with relative
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

#Save resulting video
#imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
imageio.mimsave('/content/gdrive/My Drive/Colab Data/first-order-motion-model/result_relative.mp4', [img_as_ubyte(frame) for frame in predictions])

#Show result
HTML(display(source_image, driving_video, predictions).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [ ]:
#Perform image animation with non relative
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)

#Save resulting video
imageio.mimsave('/content/gdrive/My Drive/first-order-motion-model/result_nonralative.mp4', [img_as_ubyte(frame) for frame in predictions])

#Show result
HTML(display(source_image, driving_video, predictions).to_html5_video())

## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [ ]:
#Crop video
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [ ]:
#Load image and driving video
source_image = imageio.imread('/content/gdrive/My Drive/Colab Data/image/01.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/Colab Data/video/01.mp4', memtest=False)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

#Perform image animation with relative
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

#Save resulting video
imageio.mimsave('/content/gdrive/My Drive/Colab Data/result.mp4', [img_as_ubyte(frame) for frame in predictions])

#Show result
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 191/191 [00:25<00:00,  7.39it/s]


Full Project with high quality

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

#Display result
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani   

#Create a model and load checkpoints
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Colab Data/first-order-motion-model/vox-cpk.pth.tar')

#Load image and driving video
source_image = imageio.imread('/content/gdrive/My Drive/Colab Data/image/b15.jpg')
driving_video = imageio.mimread('/content/gdrive/My Drive/Colab Data/video/01.mp4', memtest=False)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

#Perform image animation with relative
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

#Save resulting video
#imageio.mimsave('/content/gdrive/My Drive/Colab Data/result_z1.mp4', [img_as_ubyte(frame) for frame in predictions])

#Show result
HTML(display(source_image, driving_video, predictions).to_html5_video())

In [4]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

#Display result
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani   

#Create a model and load checkpoints
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Colab Data/first-order-motion-model/vox-cpk.pth.tar')

#Load image and driving video
source_image = imageio.imread('/content/gdrive/My Drive/Colab Data/image/b16.jpg')
driving_video = imageio.mimread('/content/gdrive/My Drive/Colab Data/video/01.mp4', memtest=False)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

#Perform image animation with relative
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

#Save resulting video
#imageio.mimsave('/content/gdrive/My Drive/Colab Data/result_b16.mp4', [img_as_ubyte(frame) for frame in predictions])

#Show result
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 191/191 [00:07<00:00, 24.37it/s]
